In [1]:
'''
If you are running locally then 
1. reboot your local machine
2. create an environment called 'colab' using anaconda prompt
if you have a gpu
conda create -n colab python tensorflow-gpu 
if not 
conda create -n colab python tensorflow
3. to install jupyter notebook
conda install jupyter notebook
4. to go to the 'colab' environment
activate colab
5. change file path to locate this notebook and then type 'jupyter notebook'

If you use colab
1. save the data file in your google drive
2. goto colab and start running the code
'''

"\nIf you are running locally then \n1. reboot your local machine\n2. create an environment called 'colab' using anaconda prompt\nif you have a gpu\nconda create -n colab python tensorflow-gpu \nif not \nconda create -n colab python tensorflow\n3. to install jupyter notebook\nconda install jupyter notebook\n4. to go to the 'colab' environment\nactivate colab\n5. change file path to locate this notebook and then type 'jupyter notebook'\n\nIf you use colab\n1. save the data file in your google drive\n2. goto colab and start running the code\n"

In [2]:
#install prominent libraries with specific versions

#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4-tf
#!pip install pandas==0.25.1
#!pip install sklearn==0.21.3
#!pip install matplotlib==3.2.1
#!pip install hyperas
#!pip install hyperopt

In [3]:
from __future__ import print_function                                                                                                                                                                                                                                                                                                                              # from tensorflow.contrib.rnn import *import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Input, Activation, Flatten, BatchNormalization, Reshape,Concatenate
from keras.callbacks import ModelCheckpoint, TensorBoard, History
from keras.models import Model, load_model
from keras.layers.advanced_activations import LeakyReLU
from keras import regularizers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from matplotlib import style
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils import np_utils
from sklearn.preprocessing import*
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils import np_utils, to_categorical
import pandas as pd
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from collections import Counter
import operator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import itertools
import hyperas
import hyperopt
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe, rand
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D

C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\f

In [4]:
#Get library versions
print("tensorflow.__version__ = ", tf.__version__)
print("keras.__version__ = ", keras.__version__)
import sklearn 
print("sklearn.__version__ = ", sklearn.__version__)
print("numpy.__version__ = ", np.__version__)
print("pandas.__version__ = ", pd.__version__)
import matplotlib
print("matplotlib.__version__ = ", matplotlib.__version__)

tensorflow.__version__ =  1.10.0
keras.__version__ =  2.1.6-tf
sklearn.__version__ =  0.21.3
numpy.__version__ =  1.17.0
pandas.__version__ =  1.0.1
matplotlib.__version__ =  3.1.3


In [5]:
#random seed to generate reproduceable results
from numpy.random import seed
seed(56)
from tensorflow import set_random_seed
set_random_seed(56)
random.seed(56)
os.environ['PYTHONHASHSEED']=str(1)

In [6]:
# create these folders if they do not exist
def build_path(dirName):
    try:
        os.makedirs(dirName)    
        print("Directory " , dirName ,  " Created ")
    except:
        print("Directory " , dirName ,  " already exists")  

In [7]:
# to check if GPU is getting used locally.....you need to see CPU as well as GPU in the output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18391556768239439361
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7134468506
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11266260673548420115
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
def data():
  i = 7 # the label target - number of days to predict from the input date
  p = 7 #Number of days for target calculated in the data set
  batch_size=32
  CLASSES = 2
  time_steps = 7
  ticker='^GSPC'
    
  # read data
  df=pd.read_csv('../^GSPC_7_days_0_return_dtw.csv', index_col = 0, parse_dates = True)
    
  #add additional rolling mean data  
  rm_window =30
  rolling_mean = []
  for a in range(2,rm_window+1):
    df[ticker+'rm_'+str(a)] = df[ticker].rolling(window=rm_window,center=False).mean()
    rolling_mean.append(ticker+'rm_'+str(a))
    
  # create label
  targets=pd.DataFrame([])
  for j in range (1, p+1):
    targets=targets.append(df[ticker+'_{}d_target'.format(j)])
    targets=targets.append(df[ticker+'_{}d'.format(j)])
  targets=targets.T
  df=df.drop(targets.columns, axis=1)
  df=df[rm_window:-i]
  targets=targets[rm_window:-i]
  y=targets['^GSPC_{}d_target'.format(i)]

  #check for NaN and remove
  df.isna().mean().sum()
  y.isna().mean().sum()
  remove_list=[]
  for i in df.isnull().any().iteritems():
    if i[1] == True:
      remove_list.append(i[0])
  df=df.drop(remove_list, axis=1)
  df.isnull().any().mean()
 
  # add percent change
  df=df.pct_change()
  df=df.replace([np.inf, -np.inf],np.nan) 
  df.fillna(0, inplace=True)
  df.isnull().any().mean()
    
  # apply preprocessing 
  x_scaler=RobustScaler()
  x = x_scaler.fit_transform(df)
  # x_pred = x_scaler.fit_transform(x_pred)
  del df
  y=y.values
    
  # apply time steps
  def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
      v = X[i:(i + time_steps)]
      Xs.append(v)
      ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
  x, y = create_dataset(x, y, time_steps)

  # create train and test dataset
  x_train, x_test, y_train, y_test=train_test_split(x,y, train_size=0.7, random_state=54)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  #y_train = y_train.astype('float32')
  #y_test = y_test.astype('float32')
  y_train = np_utils.to_categorical(y_train, CLASSES, dtype='float32')
  y_test = np_utils.to_categorical(y_test, CLASSES, dtype='float32')
    
  # adjustment for batch_size
  train_start = x_train.shape[0]%batch_size
  test_start = x_test.shape[0]%batch_size
  x_train = x_train[train_start:]
  y_train = y_train[train_start:]
  x_test = x_test[test_start:]
  y_test = y_test[test_start:]

  return x_train, x_test, y_train, y_test, batch_size

In [9]:
def create_lstm_model(x_train, y_train, x_test, y_test, batch_size):
    model = Sequential()
   
    model.add(LSTM(units = {{choice([32,64,128,254])}},return_sequences = True, batch_input_shape = (batch_size,x_train.shape[1], x_train.shape[2]),
                        kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}}),
                        bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}})
                        ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(LSTM(units = {{choice([16,32,64,128])}},return_sequences = False,
                        kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}}),
                        bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}})))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))  

    #Add the output layer
    model.add(Dense(units=2))
    model.add(Activation('softmax'))
    model.summary()
   
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = Adam(lr={{choice([1.0, 0.1,0.01,0.001])}}))

    result = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=10,
                    verbose=0,validation_data=(x_test, y_test))

    validation_acc = np.amax(result.history['val_acc']) 


    print('Best validation acc of epoch:', result.history['val_acc'])
    print('Train acc of epoch:', result.history['acc'])
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [10]:
if __name__ == '__main__':
    x_train, x_test, y_train, y_test, batch_size  = data()
    print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
    
    best_run, best_model = optim.minimize(model=create_lstm_model, data=data ,algo=tpe.suggest, 
                                          max_evals=20,trials=Trials(), notebook_name='3.0 LSTM',rseed=1, verbose=False)
    print("Evaluation of best performing model:")
    best_model.save("LSTM_bestmodel.h5")
    print(best_model.get_config())
    test_score, test_accuracy = best_model.evaluate(x_test, y_test, batch_size=batch_size)
    print('test_score: ', test_score, ' test_accuracy: ', test_accuracy)
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

(8832, 7, 2782) (8832, 2) (3776, 7, 2782) (3776, 2)
_________________________________________________________________                                                      
Layer (type)                 Output Shape              Param #                                                         
lstm_1 (LSTM)                (32, 7, 32)               360320                                                          
_________________________________________________________________                                                      
batch_normalization_1 (Batch (32, 7, 32)               128                                                             
_________________________________________________________________                                                      
activation_1 (Activation)    (32, 7, 32)               0                                                               
_________________________________________________________________                                           

lstm_5 (LSTM)                (32, 7, 64)               728832                                                          
_________________________________________________________________                                                      
batch_normalization_5 (Batch (32, 7, 64)               256                                                             
_________________________________________________________________                                                      
activation_7 (Activation)    (32, 7, 64)               0                                                               
_________________________________________________________________                                                      
dropout_5 (Dropout)          (32, 7, 64)               0                                                               
_________________________________________________________________                                                      
lstm_6 (LSTM)                (32, 64)   

_________________________________________________________________                                                      
activation_13 (Activation)   (32, 7, 128)              0                                                               
_________________________________________________________________                                                      
dropout_9 (Dropout)          (32, 7, 128)              0                                                               
_________________________________________________________________                                                      
lstm_10 (LSTM)               (32, 64)                  49408                                                           
_________________________________________________________________                                                      
batch_normalization_10 (Batc (32, 64)                  256                                                             
________________________________________

dropout_13 (Dropout)         (32, 7, 254)              0                                                               
_________________________________________________________________                                                      
lstm_14 (LSTM)               (32, 32)                  36736                                                           
_________________________________________________________________                                                      
batch_normalization_14 (Batc (32, 32)                  128                                                             
_________________________________________________________________                                                      
activation_20 (Activation)   (32, 32)                  0                                                               
_________________________________________________________________                                                      
dropout_14 (Dropout)         (32, 32)   

_________________________________________________________________                                                      
batch_normalization_18 (Batc (32, 128)                 512                                                             
_________________________________________________________________                                                      
activation_26 (Activation)   (32, 128)                 0                                                               
_________________________________________________________________                                                      
dropout_18 (Dropout)         (32, 128)                 0                                                               
_________________________________________________________________                                                      
dense_9 (Dense)              (32, 2)                   258                                                             
________________________________________

...

_________________________________________________________________                                                      
dense_13 (Dense)             (32, 2)                   66                                                              
_________________________________________________________________                                                      
activation_39 (Activation)   (32, 2)                   0                                                               
Total params: 3,123,538                                                                                                
Trainable params: 3,122,966                                                                                            
Non-trainable params: 572                                                                                              
_________________________________________________________________                                                      
Best validation acc of epoch:           

activation_45 (Activation)   (32, 2)                   0                                                               
Total params: 3,168,658                                                                                                
Trainable params: 3,168,022                                                                                            
Non-trainable params: 636                                                                                              
_________________________________________________________________                                                      
Best validation acc of epoch:                                                                                          
[0.5558792372881356, 0.5558792372881356, 0.5558792372881356, 0.5558792372881356, 0.5558792372881356, 0.5558792372881356, 0.4441207627118644, 0.5558792372881356, 0.4441207627118644, 0.5558792372881356]
Train acc of epoch:                                                            

Trainable params: 1,511,426                                                                                            
Non-trainable params: 320                                                                                              
_________________________________________________________________                                                      
Best validation acc of epoch:                                                                                          
[0.5558792372881356, 0.5558792372881356, 0.5558792372881356, 0.5558792372881356, 0.4441207627118644, 0.4441207627118644, 0.4441207627118644, 0.4441207627118644, 0.5558792372881356, 0.5558792372881356]
Train acc of epoch:                                                                                                    
[0.5596693840579711, 0.5722373188405797, 0.5707653985507246, 0.5709918478260869, 0.5651041666666666, 0.5732563405797102, 0.5700860507246377, 0.5700860507246377, 0.571671195652174, 0.5698596014492754]

Best validation acc of epoch:                                                                                          
[0.4441207627118644, 0.5579978813559322, 0.5558792372881356, 0.5558792372881356, 0.4441207627118644, 0.5558792372881356, 0.5558792372881356, 0.4441207627118644, 0.4441207627118644, 0.4441207627118644]
Train acc of epoch:                                                                                                    
[0.524796195652174, 0.5275135869565217, 0.5280797101449275, 0.5184556159420289, 0.5267210144927537, 0.526947463768116, 0.5460824275362319, 0.5420063405797102, 0.5395153985507246, 0.5413269927536232]
_________________________________________________________________                                                      
Layer (type)                 Output Shape              Param #                                                         
lstm_39 (LSTM)               (32, 7, 32)               360320                                                          


{'name': 'sequential_4', 'layers': [{'class_name': 'LSTM', 'config': {'name': 'lstm_7', 'trainable': True, 'batch_input_shape': (32, 7, 2782), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'units': 64, 'activation': 'tanh', 'recurrent_activation': 'hard_sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': {'class_name': 'L1L2', 'config': {'l1': 0.0, 'l2': 0.10000000149011612}}, 'recurrent_regularizer': None, 'bias_regularizer': {'class_name': 'L1L2', 'config': {'l1': 0.05999999865889549, 'l2': 0.0}}, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_